# Week 5 Assignment - Christina Morgenstern

This assignment deals with the calculation of the Root-Mean-Square Deviation (RMSD) and the alignment of a Structure Ensemble.

In [4]:
# import libraries for assignment
import numpy as np
from math import sqrt

### 1. Calculation of sum of square values along the spatial axis

In [ ]:
## code from the book - doesn´t work

# get differences in coordinates between two structure objects atomA and atom B
deltas = atomA.coords - atomB.coords
# calculation of sum of squares using the dot product of the deltas with itself
sumSquares = dot(deltas, deltas)
# apply square root function ot get distance
distance = sqrt(sumSquares)

### Calculate sum of squares using own calculation

In [27]:
# initiate random array for atomA
atomA = np.random.rand(10,3)
atomA

array([[0.87311337, 0.4650483 , 0.02175383],
       [0.34050416, 0.20652079, 0.43397447],
       [0.40135528, 0.91681165, 0.56559819],
       [0.03679406, 0.07542399, 0.65926746],
       [0.24690185, 0.13079099, 0.29890612],
       [0.5143671 , 0.29399035, 0.95450672],
       [0.64505015, 0.68806958, 0.01331112],
       [0.11266619, 0.73103454, 0.57206961],
       [0.08361797, 0.71545553, 0.28093645],
       [0.33215566, 0.86539733, 0.60528015]])

In [28]:
# initiate random array for atomB
atomB = np.random.rand(10,3)
atomB

array([[0.70720472, 0.01642684, 0.89501349],
       [0.5404175 , 0.20331662, 0.14040896],
       [0.25989247, 0.29805651, 0.0728992 ],
       [0.39520778, 0.99203935, 0.14326951],
       [0.18263977, 0.25253189, 0.78211407],
       [0.79323247, 0.66159271, 0.85991583],
       [0.25583227, 0.3421129 , 0.01677292],
       [0.19423381, 0.66110968, 0.64549122],
       [0.92772966, 0.11058382, 0.7960151 ],
       [0.75941703, 0.27452772, 0.66924878]])

In [29]:
# calculate the differences
deltas = atomA - atomB
deltas

array([[ 0.16590866,  0.44862146, -0.87325966],
       [-0.19991334,  0.00320418,  0.29356551],
       [ 0.1414628 ,  0.61875514,  0.49269899],
       [-0.35841372, -0.91661536,  0.51599795],
       [ 0.06426208, -0.1217409 , -0.48320795],
       [-0.27886537, -0.36760236,  0.09459088],
       [ 0.38921788,  0.34595667, -0.00346179],
       [-0.08156762,  0.06992487, -0.07342161],
       [-0.84411169,  0.6048717 , -0.51507865],
       [-0.42726137,  0.5908696 , -0.06396863]])

In [30]:
# calculate the sum of squares 
sumSquares = np.sum(deltas, axis=0)
sumSquares

array([-1.42928168,  1.27624501, -0.61554496])

### 2. For loops to get summed values of all and then give a single value

In [44]:
def calcRmsds(refCoords, allCoords, weights) :
    
    """Function that calculates the RMSD.
    
    Input arguments:
    
    refCoords: array of reference coordinates
    allCoords: list of other coordinate array to compare with
    weights: list of weights
    
    Returns RMSD values for individual atoms (rmsd) 
    and each atom's RMSD over the whole set of conformations (atomRmsds)"""
    
    
    rmsds = []
    totalWeight = sum(weights)
    totalSquares = zeros(refCoords.shape)
    
    for coords in allCoords :
        
        delta = coords-refCoords
        squares = delta*delta
        totalSquares += squares
        sumSquares = weights*squares.sum(axis=1)
        rmsds.append(sqrt(sum(sumSquares)/totalWeight))
    
    nStruct = len(allCoords)
    
    atomRmsds = sqrt(totalSquares.sum(axis=1)/nStruct)
    
    return rmsds, atomRmsds

### 3. Alignment of all coordinate arrays to reference set

In [45]:
def superimposeCoords(allCoords, weights, threshold=5.0) :
    
    """Function that superimposes more than two coordinate arrays.
     
    Input arguments:
    
    allCoords: list of coordinate arrays
    weights: list of weights
    threshold: threshold value used to scale the RMDS
    
    Returns RMSD values for individual atoms (rmsd) 
    and each atom's RMSD over the whole set of conformations (atomRmsds)"""
        
    nStruct = len(allCoords)

    refCoords = allCoords[0]
    meanCoords = zeros(refCoords.shape)
    rotations = []

    for index, coords in enumerate(allCoords) :
        
        if index == 0:
            rotation = identity(3)
    
    else:  
        
        rotation, coords = alignCoords(refCoords, coords, weights)
        allCoords[index] = coords 
      
    rotations.append(rotation)
    meanCoords += coords
    
    meanCoords /= nStruct
  
    rmsds, atomRmsds = calcRmsds(meanCoords, allCoords, weights)  
  
    bestRmsd = min(rmsds)
    bestIndex = rmsds.index(bestRmsd)
    bestCoords = allCoords[bestIndex]

    weightScale = atomRmsds/threshold
    weights *= exp(-weightScale*weightScale)
  
    meanCoords = bestCoords.copy()
  
    for index, coords in enumerate(allCoords) :
        
        if index != bestIndex :
            
            rotation, coords = alignCoords(bestCoords, coords, weights)
            rotations[index] = rotation
            allCoords[index] = coords 
            meanCoords += coords
      
    meanCoords /= nStruct     
    weights = ones(len(weights))
    rmsds, atomRmsds = calcRmsds(meanCoords, allCoords, weights)
  
    return allCoords, rmsds, atomRmsds, rotations